In [14]:
!pip install web3 pandas py-solc-x

In [15]:
import pandas as pd
from web3 import Web3
from solcx import compile_source, install_solc
import re
import time

In [16]:
# Install compiler
install_solc('0.8.0')

<Version('0.8.0')>

In [17]:
# Connect to Ganache
ganache_url = "HTTP://127.0.0.1:7545"
web3 = Web3(Web3.HTTPProvider(ganache_url))
web3.eth.default_account = web3.eth.accounts[0]
print("Connected to Ganache:", web3.is_connected())

Connected to Ganache: True


In [18]:
# Define and deploy contract
contract_source = '''
// SPDX-License-Identifier: MIT
pragma solidity ^0.8.0;

contract IoTDataStorage {
    struct DataRecord {
        uint256 timestamp;
        string sensorID;
        string soil_moisture;
        string electrical_conductivity;
        string soil_ph;
        string sensorTime;
    }
    
    DataRecord[] public dataRecords;
    
    function storeData(
        string memory _sensorID,
        string memory _soil_moisture,
        string memory _electrical_conductivity,
        string memory _soil_ph,
        string memory _sensorTime
    ) public {
        dataRecords.push(DataRecord(
            block.timestamp,
            _sensorID,
            _soil_moisture,
            _electrical_conductivity,
            _soil_ph,
            _sensorTime
        ));
    }
    
    function getRecord(uint index) public view returns (
        uint256, string memory, string memory, string memory, 
        string memory, string memory
    ) {
        DataRecord memory r = dataRecords[index];
        return (
            r.timestamp, 
            r.sensorID, 
            r.soil_moisture,
            r.electrical_conductivity,
            r.soil_ph,
            r.sensorTime
        );
    }
    
    function getTotalRecords() public view returns (uint) {
        return dataRecords.length;
    }
}
'''

In [19]:
# Compile and deploy
compiled = compile_source(contract_source, solc_version='0.8.0')
contract_id, contract_interface = compiled.popitem()
bytecode = contract_interface['bin']
abi = contract_interface['abi']

In [20]:
# Deploy with sufficient gas
contract = web3.eth.contract(abi=abi, bytecode=bytecode)
tx_hash = contract.constructor().transact({'gas': 3000000})
tx_receipt = web3.eth.wait_for_transaction_receipt(tx_hash)
contract_address = tx_receipt.contractAddress

print(f"✅ Contract deployed at {contract_address}")
print(f"ABI: {abi}")

✅ Contract deployed at 0xa44223159ee4B6a2B87bF79577F97b99CF2f1a36
ABI: [{'inputs': [{'internalType': 'uint256', 'name': '', 'type': 'uint256'}], 'name': 'dataRecords', 'outputs': [{'internalType': 'uint256', 'name': 'timestamp', 'type': 'uint256'}, {'internalType': 'string', 'name': 'sensorID', 'type': 'string'}, {'internalType': 'string', 'name': 'soil_moisture', 'type': 'string'}, {'internalType': 'string', 'name': 'electrical_conductivity', 'type': 'string'}, {'internalType': 'string', 'name': 'soil_ph', 'type': 'string'}, {'internalType': 'string', 'name': 'sensorTime', 'type': 'string'}], 'stateMutability': 'view', 'type': 'function'}, {'inputs': [{'internalType': 'uint256', 'name': 'index', 'type': 'uint256'}], 'name': 'getRecord', 'outputs': [{'internalType': 'uint256', 'name': '', 'type': 'uint256'}, {'internalType': 'string', 'name': '', 'type': 'string'}, {'internalType': 'string', 'name': '', 'type': 'string'}, {'internalType': 'string', 'name': '', 'type': 'string'}, {'inte

In [21]:
# Data cleaning and storage functions
def clean_value(value_str):
    """Extract numeric value from strings like '30%' or '[8.7]'"""
    match = re.search(r'(\d+\.?\d*)', str(value_str))
    return match.group(1) if match else "0"

In [22]:
# Load and clean data
df = pd.read_csv("iot_data_soil.csv")
print("Original data:")
print(df.head())

Original data:
  Sensor ID Soil Moisture (%) Electrical Conductivity (dS/m) Soil pH  \
0  Sensor_1               30%                          [8.7]  [11.4]   
1  Sensor_2               29%                           [1.]  [12.4]   
2  Sensor_3                4%                          [0.9]    [3.]   
3  Sensor_4               31%                          [5.7]  [13.9]   
4  Sensor_5               49%                          [9.4]   [3.7]   

             Timestamp  
0  2025-05-23 19:57:42  
1  2025-05-24 01:33:42  
2  2025-05-24 04:45:42  
3  2025-05-23 18:31:42  
4  2025-05-24 13:16:42  


In [23]:
# Clean values
df['Soil Moisture (%)'] = df['Soil Moisture (%)'].apply(clean_value)
df['Electrical Conductivity (dS/m)'] = df['Electrical Conductivity (dS/m)'].apply(clean_value)
df['Soil pH'] = df['Soil pH'].apply(clean_value)

print("\nCleaned data:")
print(df.head())


Cleaned data:
  Sensor ID Soil Moisture (%) Electrical Conductivity (dS/m) Soil pH  \
0  Sensor_1                30                            8.7    11.4   
1  Sensor_2                29                             1.    12.4   
2  Sensor_3                 4                            0.9      3.   
3  Sensor_4                31                            5.7    13.9   
4  Sensor_5                49                            9.4     3.7   

             Timestamp  
0  2025-05-23 19:57:42  
1  2025-05-24 01:33:42  
2  2025-05-24 04:45:42  
3  2025-05-23 18:31:42  
4  2025-05-24 13:16:42  


In [24]:
# Create contract instance
contract = web3.eth.contract(address=contract_address, abi=abi)

In [25]:
# Send data
for idx, row in df.iterrows():
    tx_hash = contract.functions.storeData(
        str(row['Sensor ID']),
        str(row['Soil Moisture (%)']),
        str(row['Electrical Conductivity (dS/m)']),
        str(row['Soil pH']),
        str(row['Timestamp'])
    ).transact({'gas': 1000000})
    receipt = web3.eth.wait_for_transaction_receipt(tx_hash)
    print(f"Stored record {idx+1}/{len(df)}: {receipt.transactionHash.hex()}")
    time.sleep(0.5)  # Avoid overwhelming Ganache

print("✅ All data stored!")

Stored record 1/10: 46d92ec30ab7660dbeaf0c1936b7c5880ab2fcd4b4f8f42ae5fe212b6274755e
Stored record 2/10: 21c6ca42d0ae07766b1f87bc914121f2f084a8eb99084126ed835cb45738b7f9
Stored record 3/10: 31883e97c06256d0dae443955806962018ce2d000beb32b9d4c8da686e508da9
Stored record 4/10: 9de0f1a4607c1a84cd98cf678c4e7f5ee9c98b08358871d5ee9415d3b630c791
Stored record 5/10: 46a035cd114972a798a89b13cb4211ff2e646a670978c7faf7fbce418e163cb5
Stored record 6/10: d064a461367f53dfd02246a99c135613ad2ab70eea5d2a82f2beaab39d9bef70
Stored record 7/10: 52d3a3ea25c8e49e440cc7cd7d3d7326cd69343eb9bdd1bc6d42918962ef9621
Stored record 8/10: 92706acadd56c518c388ed6ddce3ef91bf428018c2804e56af522048acb2967b
Stored record 9/10: df2956c552fccb9ec11257e88b42728ac970249e0b2f06443edd92eda76b72f2
Stored record 10/10: f669836828462cd2e18c96f0b6c13ceb66aee8e91f33b56a321a6d051133f1b7
✅ All data stored!


In [26]:
# Retrieve and process data
total_records = contract.functions.getTotalRecords().call()
print(f"Total records retrieved: {total_records}")

Total records retrieved: 10


In [27]:
# Process into DataFrame
data = []
for i in range(total_records):
    record = contract.functions.getRecord(i).call()
    data.append({
        "block_timestamp": record[0],
        "sensor_id": record[1],
        "soil_moisture": record[2],
        "electrical_conductivity": record[3],
        "soil_ph": record[4],
        "sensor_timestamp": record[5]
    })

df_retrieved = pd.DataFrame(data)
df_retrieved["block_timestamp"] = pd.to_datetime(df_retrieved["block_timestamp"], unit="s")
df_retrieved["sensor_timestamp"] = pd.to_datetime(df_retrieved["sensor_timestamp"])

In [28]:
# Convert to numeric
df_retrieved["soil_moisture"] = pd.to_numeric(df_retrieved["soil_moisture"])
df_retrieved["electrical_conductivity"] = pd.to_numeric(df_retrieved["electrical_conductivity"])
df_retrieved["soil_ph"] = pd.to_numeric(df_retrieved["soil_ph"])

In [29]:
# Save cleaned data
df_retrieved.to_csv("cleaned_iot_data.csv", index=False)
print("✅ Cleaned data saved!")
print("\nFinal cleaned data:")
print(df_retrieved.head())

✅ Cleaned data saved!

Final cleaned data:
      block_timestamp sensor_id  soil_moisture  electrical_conductivity  \
0 2025-06-09 04:09:02  Sensor_1             30                      8.7   
1 2025-06-09 04:09:03  Sensor_2             29                      1.0   
2 2025-06-09 04:09:04  Sensor_3              4                      0.9   
3 2025-06-09 04:09:04  Sensor_4             31                      5.7   
4 2025-06-09 04:09:05  Sensor_5             49                      9.4   

   soil_ph    sensor_timestamp  
0     11.4 2025-05-23 19:57:42  
1     12.4 2025-05-24 01:33:42  
2      3.0 2025-05-24 04:45:42  
3     13.9 2025-05-23 18:31:42  
4      3.7 2025-05-24 13:16:42  
